In [ ]:
import $ivy.`org.apache.spark::spark-sql:3.5.0`

In [ ]:
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.WARN)

## Start

In [ ]:
import org.apache.spark.sql._
import org.apache.spark.sql.types.{IntegerType, StringType, StructField, StructType}
import org.apache.spark.sql.functions._

In [ ]:
val spark = SparkSession
                .builder()
                .master("local[*]")
                .appName("Dataframe API")
                .getOrCreate()

import spark.implicits._

In [ ]:
println(s"spark.version == ${spark.version}")

## 1 - Создание DataFrame

### fromRDD

In [ ]:
val columns = Seq("StudentID", "Course")
val data = Seq(("1", "Spark"), ("2", "Scala"), ("3", "Java"))

val fromRDD: DataFrame = spark.sparkContext.parallelize(data).toDF(columns: _*)

fromRDD.show

### fromList

In [ ]:
val fromList: DataFrame = data.toDF()

fromList.show

### createDataFrame

In [ ]:
val createDataFrame: DataFrame = spark.createDataFrame(data)

createDataFrame.show

### withSchema

In [ ]:
val withSchema: DataFrame = {
    val schema = StructType( Array(
      StructField("StudentID", IntegerType, true),
      StructField("Course", StringType, true)
    ))

    val rdd = (spark.sparkContext.parallelize(Seq(
      Row(1, "Spark"),
      Row(2, "Scala")
    )))

    spark.createDataFrame(rdd, schema)
}

withSchema.show

In [ ]:
withSchema.printSchema

In [ ]:
withSchema.dtypes

### fromFile

In [ ]:
val fromFile: DataFrame = spark.read.format("json").load("data/customer_data.json")

fromFile.show(5, false)

## 2 - Операции

In [ ]:
val customerDf = fromFile

### basicOperations

In [ ]:
customerDf.printSchema()

In [ ]:
customerDf.head

#### Select

In [ ]:
customerDf.select("Birthdate", "Country").show

In [ ]:
customerDf.select(col("Country")).show

In [ ]:
customerDf.select('Country).show

In [ ]:
customerDf.selectExpr("Birthdate as Date").show

In [ ]:
customerDf.withColumn("Flag", lit(true)).show

In [ ]:
customerDf.withColumnRenamed("Birthdate", "Date").show

#### Filter

In [ ]:
customerDf.filter("Country = 'Norway'").show

In [ ]:
customerDf.where('Country =!= "Iceland").show(false)

#### Sort

In [ ]:
customerDf.sort('CustomerID.desc).show

In [ ]:
customerDf.orderBy("CustomerID").show

#### Repartition

In [ ]:
println(s"Num partitions: ${customerDf.repartition(5).rdd.getNumPartitions}")

In [ ]:
println(s"Num partitions: ${customerDf.rdd.getNumPartitions}")


In [ ]:
val repartitionedDf = customerDf.repartition(5, col("Country"))
println(s"New num partitions: ${repartitionedDf.rdd.getNumPartitions}")

In [ ]:
println(s"Num partitions after coalesce: ${repartitionedDf.coalesce(1).rdd.getNumPartitions}")

### functions

In [ ]:
customerDf.select("Birthdate").show(5)

In [ ]:
customerDf.select(date_format(col("Birthdate"), "yyyy-MM-dd").alias("bd")).show(5)

In [ ]:
customerDf.select("Name", "Username").show(5)

In [ ]:
customerDf.withColumn("Identity", array('Name, 'Username)).printSchema

In [ ]:
customerDf.withColumn("Identity", array('Name, 'Username)).select("Name", "Username", "Identity").show(5, false)

### groupBy

In [ ]:
customerDf.groupBy("Country").agg(count(lit(1))).show

In [ ]:
customerDf.groupBy("Country").count().show

In [ ]:
customerDf.groupBy("Country", "Birthdate").agg(min("CustomerID"), max("CustomerID")).orderBy("Country").show

### union

In [ ]:
customerDf.count

In [ ]:
customerDf.union(customerDf).count

In [ ]:
customerDf.count * 5

In [ ]:
(1 to 5).toList.map(a => customerDf).reduce((x, y) => x.union(y)).count

### join

In [ ]:
val retailDf = spark.read.format("json").load("data/retail_data.json")

In [ ]:
customerDf.printSchema

In [ ]:
retailDf.printSchema

In [ ]:
(customerDf.dtypes.map(_._1)).toSet.intersect((retailDf.dtypes.map(_._1)).toSet)

In [ ]:
customerDf.join(retailDf, "CustomerID").show

In [ ]:
customerDf
    .join(retailDf, customerDf("CustomerID") === retailDf("CustomerID"), "left")
    //.select("CustomerID")
    .show

## Stop

In [ ]:
spark.stop()